In [ ]:
df["Monthly Mean Total Sunspot Number"][len(df["Monthly Mean Total Sunspot Number"]):].plot.line()

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor

# 🔹 Veri setini oku
df = pd.read_csv('Sunspots.csv')
df.drop(columns='Unnamed: 0', inplace=True)

# 🔹 Sıfır olanları çıkar
df.drop(index=df[df["Monthly Mean Total Sunspot Number"] == 0].index, inplace=True)

# 🔹 Tarih formatı ve yeni özellikler
df['Date'] = pd.to_datetime(df['Date'])
df['Sunspot_lag1'] = df['Monthly Mean Total Sunspot Number'].shift(1)
df['Diff_11Year'] = df['Monthly Mean Total Sunspot Number'].diff(periods=11)
df['Diff_1Year'] = df['Monthly Mean Total Sunspot Number'].diff(periods=1)
df['Month'] = df['Date'].dt.month
df['Month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)
df['Year'] = df['Date'].dt.year

# 🔹 Boş verileri çıkar
df.dropna(inplace=True)

# 🔹 Özellikler ve hedef değişken
features = [
    'Sunspot_lag1',
    'Month',
    'Diff_11Year',
    'Diff_1Year',
    'Month_sin',
    'Month_cos',
    'Year'
]
X = df[features]
y = df['Monthly Mean Total Sunspot Number']

# 🔹 Eğitim-test bölünmesi
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 🔹 Ölçekleme
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 🔹 KNN modeli
model_knn = KNeighborsRegressor(n_neighbors=5)
model_knn.fit(X_train_scaled, y_train)

# 🔹 Tahmin
y_pred = model_knn.predict(X_test_scaled)

# 🔹 Performans metrikleri
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# 🔹 Sonuçları yazdır
print("🔹 KNN")
print(f"R² Score: {r2:.4f}")
print(f"MSE: {mse:.2f}")
print(f"MAE: {mae:.2f}")


In [ ]:
a=X[-20:]

In [ ]:
a

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# 🔹 Tarih bilgilerini eşleştir
date_test = df.loc[y_test.index, 'Date']

# 🔹 DataFrame oluştur
plot_df = pd.DataFrame({
    'Date': date_test,
    'Actual': y_test,
    'Predicted': y_pred
})

# 🔹 Tarihe göre sırala
plot_df.sort_values('Date', inplace=True)

# 🔹 (Opsiyonel) 2000 sonrası filtre
plot_df = plot_df[plot_df['Date'] >= '2000-01-01']

# 🔹 Grafik çizimi
plt.figure(figsize=(14, 6))
plt.plot(plot_df['Date'], plot_df['Actual'], label='Actual Sunspots', color='black', alpha=0.6)
plt.plot(plot_df['Date'], plot_df['Predicted'], label='KNN Prediction', color='royalblue', linestyle='--')
plt.xlabel('Date')
plt.ylabel('Monthly Mean Sunspot Number')
plt.title('KNN Prediction vs Actual Sunspots (2000 and after, sorted)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# 🔹 Tarihleri eşleştir
date_test = df.loc[y_test.index, 'Date']

# 🔹 DataFrame oluştur ve sırala
plot_df = pd.DataFrame({
    'Date': date_test,
    'Actual': y_test,
    'Predicted': y_pred
}).sort_values('Date')

# 🔹 Hata sütunu ekle
plot_df['Error'] = plot_df['Predicted'] - plot_df['Actual']

# 🔹 Grafik 1: Tahmin vs Gerçek (daha önceki)
plt.figure(figsize=(14, 6))
cut_index = int(len(plot_df) * 0.8)
plt.plot(plot_df['Date'], plot_df['Actual'], label='Actual Sunspots', color='yellow', alpha=0.6)
plt.plot(plot_df['Date'].iloc[cut_index:], plot_df['Predicted'].iloc[cut_index:], 
         label='KNN Prediction (last 20%)', color='black', linestyle='--')
plt.xlabel('Date')
plt.ylabel('Monthly Mean Sunspot Number')
plt.title('KNN Prediction vs Actual Sunspots')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# 🔹 Grafik 2: Hata Grafiği (Error Plot)
plt.figure(figsize=(14, 4))
plt.plot(plot_df['Date'], plot_df['Error'], color='crimson')
plt.axhline(y=0, color='gray', linestyle='--')
plt.xlabel('Date')
plt.ylabel('Prediction Error')
plt.title('Prediction Error Over Time (KNN)')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import pandas as pd
from datetime import timedelta

# 🔹 Kopya dataframe, son bilinen veriyle başla
forecast_months = 24  # 5 yıl
future_predictions = []

# Son bilinen kayıt
last_row = df.sort_values('Date').iloc[-1].copy()

for _ in range(forecast_months):
    # 🔹 Yeni tarih
    last_date = last_row['Date']
    new_date = last_date + pd.DateOffset(months=1)

    # 🔹 Yeni ay, yıl
    new_month = new_date.month
    new_year = new_date.year

    # 🔹 Yeni cyclic özellikler
    new_month_sin = np.sin(2 * np.pi * new_month / 12)
    new_month_cos = np.cos(2 * np.pi * new_month / 12)

    # 🔹 Yeni diff değerleri bilinemez; 0 kabul edebiliriz ya da sabit
    new_diff_11 = 0
    new_diff_1 = 0

    # 🔹 Girdi vektörü oluştur
    X_new = pd.DataFrame([{
        'Sunspot_lag1': last_row['Monthly Mean Total Sunspot Number'],
        'Month': new_month,
        'Diff_11Year': new_diff_11,
        'Diff_1Year': new_diff_1,
        'Month_sin': new_month_sin,
        'Month_cos': new_month_cos,
        'Year': new_year
    }])

    # 🔹 Ölçekle
    X_new_scaled = scaler.transform(X_new)

    # 🔹 Tahmin et
    y_pred_future = model_knn.predict(X_new_scaled)[0]

    # 🔹 Tahmini ve bilgileri kaydet
    future_predictions.append({
        'Date': new_date,
        'Predicted Sunspot': y_pred_future
    })

    # 🔹 Yeni tahmini bir sonraki iterasyonda lag olarak kullanmak üzere sakla
    last_row['Date'] = new_date
    last_row['Monthly Mean Total Sunspot Number'] = y_pred_future

# 🔹 Sonuçları DataFrame'e çevir
future_df = pd.DataFrame(future_predictions)


In [ ]:
plt.figure(figsize=(14, 6))

# 🔹 Gerçek veriler (son 10 yıl)
recent_actual = df[df['Date'] >= df['Date'].max() - pd.DateOffset(years=5)]
plt.plot(recent_actual['Date'], recent_actual['Monthly Mean Total Sunspot Number'], 
         label='Actual (Last 10 Years)', color='black')

# 🔹 Gelecek tahminler
plt.plot(future_df['Date'], future_df['Predicted Sunspot'], 
         label='KNN Forecast (Next 5 Years)', color='blue', linestyle='--')

plt.xlabel('Date')
plt.ylabel('Monthly Mean Sunspot Number')
plt.title('KNN Forecast: Next 5 Years of Sunspots')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
